In [1]:
import numpy as np

c = np.array([[1,0,0,0,0,0,0]])#입력(2차원), 원-핫 표현
W = np.random.randn(7,3)#가중치
h = np.matmul(c,W)#중간 노드
print(h)

[[-0.18545942 -0.76276988  0.74948955]]


In [2]:
#Matmul계층으로 수행
import sys
sys.path.append('..')
from common.layers import MatMul

c = np.array([[1,0,0,0,0,0,0]])#입력
W = np.random.randn(7,3)#가중치
layer = MatMul(W)#Matmul 계층
h = layer.forward(c)#순전파 수행
print(h)

[[-0.46922656  0.07682951  1.20890558]]
